# Scraping techniques to obtain prices

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from googletrans import Translator
import numpy as np

## Load dataset

In [223]:
file_path = 'df_off_spain_price.pkl'
df_off_spain = pd.read_pickle(file_path)

### Normalize brand names

In [224]:
df_off_spain["brands"].value_counts().head(50)

brands
Bonarea                    217
Mercadona                   46
                            32
Carrefour                   21
bonarea                     20
Realvalle                   14
Juan Luna                   10
Navidul                      9
mercadona                    8
Auchan                       8
boadas                       7
juan luna                    7
Alteza                       6
Noel                         6
BonÁrea                      6
Froiz                        6
Elpozo                       6
Espuña                       6
Dia                          6
Argal                        5
Hiper Dino                   5
nico                         5
Covap                        4
El Corte Inglés              4
Hacendado                    4
BonÀrea                      4
Consum                       4
costa brava                  3
Arrullo                      3
Spar                         3
incarlopsa                   3
arroyo                       3
s

In [225]:
def normalize_text(text):
    if isinstance(text, str) and text.strip() != "":
        return unidecode(text).lower()
    return text

df_off_spain['norm_product_name'] = df_off_spain['product_name'].apply(normalize_text)
df_off_spain['brands'] = df_off_spain['brands'].apply(normalize_text)

In [226]:
df_off_spain["brands"].value_counts().head(50)

brands
bonarea                    250
mercadona                   54
                            32
carrefour                   22
juan luna                   17
realvalle                   14
boadas                      10
auchan                      10
navidul                      9
espuna                       9
elpozo                       8
noel                         8
froiz                        7
nico                         6
alteza                       6
incarlopsa                   6
sanchez alcaraz              6
dia                          6
redondo iglesias             5
argal                        5
covap                        5
consum                       5
hiper dino                   5
la hacienda del iberico      4
supersol                     4
el corte ingles              4
hacendado                    4
arroyo                       4
arrullo                      4
campo de los nogales         4
la hoguera                   4
campodulce                   3
s

## Scraping Carrefour

In [20]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-gpu")

chrome_driver_path = "chromedriver.exe"

driver = webdriver.Chrome(service=ChromeService(executable_path=chrome_driver_path), options=chrome_options)

EANs = df_off_spain["code"].values

df_carrefour = pd.DataFrame(columns=["code", "scrap_price/100g"])

first_time = True
for indx, ean in enumerate(EANs):
    indx += 1
    url = f"https://www.carrefour.es/?q={ean}"
    driver.get(url)
    
    
    if first_time:
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )
        button.click()
        first_time = False
        
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    time.sleep(3)
    try:
        no_results_element = driver.find_element(By.CLASS_NAME, 'ebx-no-results__message')
        print(f"EAN: {ean} no encontrado... {indx}/{len(EANs)}.")
    except NoSuchElementException:
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ebx-grid"]')))
            time.sleep(1)
            first_product = driver.find_element(By.XPATH, '//*[@id="ebx-grid"]').find_element(By.XPATH, '*')
            price = first_product.find_element(By.CLASS_NAME, 'ebx-result__quantity.ebx-result-quantity').text
            unit = "€/kg"
            if unit in price:
                price = price.replace(" "+unit, "").replace(",", ".")
                price = float(price)/10
            print(f"** Producto encontrado! EAN: {ean} ** {indx}/{len(EANs)}.")
            df_carrefour = pd.concat([df_carrefour, pd.DataFrame([{
                "code": ean,
                "scrap_price/100g": price
            }])], ignore_index=True)
        except:
            print(f"strange error on {ean}")
            continue
driver.quit()

EAN: 20415136 no encontrado... 1/1330.
EAN: 20526641 no encontrado... 2/1330.
EAN: 8410320031462 no encontrado... 3/1330.
EAN: 8410320242004 no encontrado... 4/1330.
EAN: 8410468001372 no encontrado... 5/1330.
EAN: 8421384012014 no encontrado... 6/1330.
EAN: 8414807512499 no encontrado... 7/1330.
EAN: 8421384009724 no encontrado... 8/1330.
EAN: 8425545113411 no encontrado... 9/1330.
** Producto encontrado! EAN: 8431876142921 ** 10/1330.


C:\Users\oscar\AppData\Local\Temp\ipykernel_10660\2962916694.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_carrefour = pd.concat([df_carrefour, pd.DataFrame([{


EAN: 8436030021005 no encontrado... 11/1330.
EAN: 8433329079266 no encontrado... 12/1330.
EAN: 8437007350944 no encontrado... 13/1330.
EAN: 8480000582287 no encontrado... 14/1330.
EAN: 8480017077165 no encontrado... 15/1330.
EAN: 8437005247864 no encontrado... 16/1330.
EAN: 8480000582294 no encontrado... 17/1330.
EAN: 8410320287449 no encontrado... 18/1330.
EAN: 20188016 no encontrado... 19/1330.
EAN: 20077792 no encontrado... 20/1330.
EAN: 8480017075017 no encontrado... 21/1330.
EAN: 24026680 no encontrado... 22/1330.
EAN: 00569008500119002642 no encontrado... 23/1330.
EAN: 8410320131797 no encontrado... 24/1330.
EAN: 8431945708270 no encontrado... 25/1330.
EAN: 8425545117624 no encontrado... 26/1330.
EAN: 20078669 no encontrado... 27/1330.
EAN: 20077822 no encontrado... 28/1330.
EAN: 8436037457920 no encontrado... 29/1330.
EAN: 8436011771912 no encontrado... 30/1330.
EAN: 8437011582850 no encontrado... 31/1330.
EAN: 20797256 no encontrado... 32/1330.
EAN: 8410764010474 no encontrado.

EAN: 8410783329311 no encontrado... 187/1330.
EAN: 8480024640239 no encontrado... 188/1330.
EAN: 8425402176719 no encontrado... 189/1330.
EAN: 230582800008812500011008 no encontrado... 190/1330.
EAN: 00584726700119003805 no encontrado... 191/1330.
EAN: 8423759050539 no encontrado... 192/1330.
EAN: 230583240011615500017981 no encontrado... 193/1330.
EAN: 8410468001402 no encontrado... 194/1330.
EAN: 8422823575428 no encontrado... 195/1330.
EAN: 8436563410086 no encontrado... 196/1330.
EAN: 8436554000524 no encontrado... 197/1330.
EAN: 00703726200119001344 no encontrado... 198/1330.
EAN: 8033075800918 no encontrado... 199/1330.
EAN: 8431945708423 no encontrado... 200/1330.
EAN: 8413387050025 no encontrado... 201/1330.
EAN: 8436018475080 no encontrado... 202/1330.
EAN: 00544724000119006728 no encontrado... 203/1330.
** Producto encontrado! EAN: 8436533735300 ** 204/1330.
EAN: 8410060487307 no encontrado... 205/1330.
EAN: 8430803030034 no encontrado... 206/1330.
EAN: 8437009078655 no encon

EAN: 2080125002156 no encontrado... 357/1330.
EAN: 8437014862683 no encontrado... 358/1330.
EAN: 8425852370859 no encontrado... 359/1330.
EAN: 00584734300219003324 no encontrado... 360/1330.
EAN: 8431945704203 no encontrado... 361/1330.
EAN: 00412532408719131278 no encontrado... 362/1330.
EAN: 2401449973517 no encontrado... 363/1330.
EAN: 00569032900119002463 no encontrado... 364/1330.
EAN: 230582800008612500010755 no encontrado... 365/1330.
EAN: 8436011771714 no encontrado... 366/1330.
EAN: 8424355024108 no encontrado... 367/1330.
EAN: 8437002975319 no encontrado... 368/1330.
EAN: 8413387510024 no encontrado... 369/1330.
EAN: 8436570450501 no encontrado... 370/1330.
EAN: 8421384105747 no encontrado... 371/1330.
EAN: 8414606550302 no encontrado... 372/1330.
EAN: 8437012739420 no encontrado... 373/1330.
EAN: 8437004674067 no encontrado... 374/1330.
EAN: 8437000796428 no encontrado... 375/1330.
EAN: 00661231108219035729 no encontrado... 376/1330.
EAN: 8426765494816 no encontrado... 377/1

EAN: 8425545117815 no encontrado... 525/1330.
EAN: 8437014971033 no encontrado... 526/1330.
EAN: 8436578484829 no encontrado... 527/1330.
EAN: 8433329108379 no encontrado... 528/1330.
EAN: 00428408500120002040 no encontrado... 529/1330.
EAN: 00569006500120002701 no encontrado... 530/1330.
EAN: 2100158001008 no encontrado... 531/1330.
EAN: 8436576950395 no encontrado... 532/1330.
EAN: 8437009203019 no encontrado... 533/1330.
EAN: 8437006642804 no encontrado... 534/1330.
EAN: 00597411604120001656 no encontrado... 535/1330.
EAN: 8435324003703 no encontrado... 536/1330.
** Producto encontrado! EAN: 49783810 ** 537/1330.
EAN: 8425545112858 no encontrado... 538/1330.
EAN: 8410060782501 no encontrado... 539/1330.
EAN: 8426262015750 no encontrado... 540/1330.
EAN: 8436554000142 no encontrado... 541/1330.
EAN: 8422598035219 no encontrado... 542/1330.
EAN: 8410783343003 no encontrado... 543/1330.
EAN: 8436557161154 no encontrado... 544/1330.
EAN: 8424916129358 no encontrado... 545/1330.
strange 

EAN: 2401643111920 no encontrado... 695/1330.
EAN: 8431945792125 no encontrado... 696/1330.
EAN: 8437005103733 no encontrado... 697/1330.
EAN: 8425545113343 no encontrado... 698/1330.
EAN: 8436570752315 no encontrado... 699/1330.
EAN: 8436548163679 no encontrado... 700/1330.
EAN: 2359135004802 no encontrado... 701/1330.
EAN: 8437002761141 no encontrado... 702/1330.
EAN: 8437013041560 no encontrado... 703/1330.
EAN: 8423610286817 no encontrado... 704/1330.
EAN: 8435324003727 no encontrado... 705/1330.
EAN: 8436037451744 no encontrado... 706/1330.
EAN: 8435115058967 no encontrado... 707/1330.
EAN: 8435115059001 no encontrado... 708/1330.
EAN: 8436023616225 no encontrado... 709/1330.
EAN: 8431695003007 no encontrado... 710/1330.
EAN: 4056489257349 no encontrado... 711/1330.
EAN: 8410843073048 no encontrado... 712/1330.
EAN: 230582790009403500003297 no encontrado... 713/1330.
EAN: 00569035600120002847 no encontrado... 714/1330.
EAN: 00428435700120002758 no encontrado... 715/1330.
EAN: 0059

EAN: 8414881410445 no encontrado... 862/1330.
EAN: 13301257 no encontrado... 863/1330.
EAN: 8421395038751 no encontrado... 864/1330.
EAN: 8437020040310 no encontrado... 865/1330.
EAN: 2004060025223 no encontrado... 866/1330.
EAN: 00748223800121001265 no encontrado... 867/1330.
EAN: 8435324003734 no encontrado... 868/1330.
EAN: 8437002032234 no encontrado... 869/1330.
EAN: 230583190010406500006766 no encontrado... 870/1330.
EAN: 8410843130697 no encontrado... 871/1330.
EAN: 230583210012803500004489 no encontrado... 872/1330.
EAN: 8422598056061 no encontrado... 873/1330.
EAN: 230583750007405000003703 no encontrado... 874/1330.
EAN: 00569025900121002871 no encontrado... 875/1330.
EAN: 230583740008812500011006 no encontrado... 876/1330.
EAN: 230582780011506500007480 no encontrado... 877/1330.
EAN: 00584725800121003662 no encontrado... 878/1330.
EAN: 00586425400121001766 no encontrado... 879/1330.
EAN: 8422598051172 no encontrado... 880/1330.
EAN: 8423248001042 no encontrado... 881/1330.
EA

EAN: 230582780008407000005883 no encontrado... 1028/1330.
EAN: 00544706900122007283 no encontrado... 1029/1330.
EAN: 8431945807508 no encontrado... 1030/1330.
EAN: 230157000016801895003188 no encontrado... 1031/1330.
EAN: 00428407300122003431 no encontrado... 1032/1330.
EAN: 8413387304692 no encontrado... 1033/1330.
EAN: 00569006700122002901 no encontrado... 1034/1330.
EAN: 00597206000122003757 no encontrado... 1035/1330.
EAN: 8421334600025 no encontrado... 1036/1330.
EAN: 8433329130806 no encontrado... 1037/1330.
EAN: 8437002975401 no encontrado... 1038/1330.
EAN: 230582780007607000005321 no encontrado... 1039/1330.
EAN: 91236028383683 no encontrado... 1040/1330.
EAN: 2405954001659 no encontrado... 1041/1330.
EAN: 230582780012207000008548 no encontrado... 1042/1330.
EAN: 230582780010807000007569 no encontrado... 1043/1330.
** Producto encontrado! EAN: 8431876105599 ** 1044/1330.
EAN: 230582780009807000006863 no encontrado... 1045/1330.
EAN: 00428408400122003635 no encontrado... 1046/1

EAN: 00597422301122001962 no encontrado... 1184/1330.
EAN: 230583190006007000004206 no encontrado... 1185/1330.
EAN: 17825829 no encontrado... 1186/1330.
EAN: 230157000015201895002883 no encontrado... 1187/1330.
EAN: 230237790016006000009603 no encontrado... 1188/1330.
EAN: 230157000012001895002271 no encontrado... 1189/1330.
EAN: 8421395011556 no encontrado... 1190/1330.
EAN: 00569025700122002968 no encontrado... 1191/1330.
** Producto encontrado! EAN: 8436043620110 ** 1192/1330.
EAN: 230582900013401850002485 no encontrado... 1193/1330.
EAN: 230582900015601850002890 no encontrado... 1194/1330.
EAN: 230228790016404900008047 no encontrado... 1195/1330.
EAN: 230228790011404900005591 no encontrado... 1196/1330.
EAN: 230157000018601895003526 no encontrado... 1197/1330.
EAN: 00569025800122003016 no encontrado... 1198/1330.
EAN: 0832693444171 no encontrado... 1199/1330.
EAN: 00681627301622002161 no encontrado... 1200/1330.
EAN: 230157010021201695003592 no encontrado... 1201/1330.
EAN: 004284

In [227]:
# show all the new prices obtained
df_carrefour

,code,scrap_price/100g
0,8431876142921,2.214
1,8436533735300,2.588
2,8431945708911,4.775
3,8410843001188,1.667
4,8431876272260,22.375
5,8436043621124,1.8
6,8436043621117,2.387
7,8410060782006,1.908
8,8410069012760,1.276
9,8431876306378,1.908


In [57]:
# special treatment for one specific product
df_carrefour = df_carrefour[df_carrefour["scrap_price/100g"] != ""].reset_index(drop=True)

special_code = "8423610262095"
index_to_replace = df_carrefour[df_carrefour['code'] == special_code].index
price = float(df_carrefour.loc[df_carrefour['code'] == special_code, 'scrap_price/100g'].values[0].split()[0].replace(",","."))
df_carrefour.loc[index_to_replace, 'scrap_price/100g'] = (price / 840) * 100

Index([14], dtype='int64')


In [228]:
# add the obtained prices to the original dataset
df_off_spain = df_off_spain.merge(df_carrefour, on='code', how='left')

## Scraping BonArea

In [97]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-gpu")

chrome_driver_path = "chromedriver.exe"

driver = webdriver.Chrome(service=ChromeService(executable_path=chrome_driver_path), options=chrome_options)

url = "https://www.bonarea-online.com/es/shop/shopping/13_300_020_060"
driver.get(url)

button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
)
button.click()


all_elements = driver.find_element(By.CLASS_NAME, 'center-div-alignement-shopping')

all_jams_bonarea = pd.DataFrame(columns=["bonarea_name", "scrap_price/100g"])
for child in all_elements.find_elements(By.XPATH, '*'):
    bonarea_name = child.find_element(By.TAG_NAME, "a").find_element(By.CLASS_NAME, "text").find_element(By.XPATH, '*').text
    price = round(float(child.find_element(By.CLASS_NAME, "price").text.split(" €/u. ")[-1][1:-1].split(" ")[0].replace(",", "."))/10, 2)
    
    all_jams_bonarea = pd.concat([all_jams_bonarea, pd.DataFrame([{
        "bonarea_name": bonarea_name,
        "scrap_price/100g": price
    }])], ignore_index=True)
    

C:\Users\oscar\AppData\Local\Temp\ipykernel_10660\2195656219.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_jams_bonarea = pd.concat([all_jams_bonarea, pd.DataFrame([{


In [230]:
# show all the available products and prices of BonArea
all_jams_bonarea

,bonarea_name,scrap_price/100g
0,Finísimo de jamón serrano,1.55
1,Jamón serrano,1.57
2,Jamón serrano medias lonchas,1.72
3,Virutas de jamón serrano,1.42
4,Jamón serrano gran reserva,1.75
5,Paleta de cebo ibérica 50% raza ibérica,4.50
6,Codillos de jamón,0.32
7,Finísimo de jamón Duroc,2.12
8,Finísimo de jamón de cebo ibérico 50% raza ibé...,3.99
9,Jamón Duroc gran selección,2.07


### Find similarity of product names

In [231]:
from fuzzywuzzy import fuzz

df1 = df_off_spain[df_off_spain["brands"]=="bonarea"][["product_name"]].reset_index(drop=True)
df2 = all_jams_bonarea.copy()

df1['norm_product_name'] = df1['product_name'].apply(normalize_text)
df2['bonarea_name'] = df2['bonarea_name'].apply(normalize_text)

def find_best_match(nombre, opciones):
    mejor_coincidencia = None
    max_ratio = 0
    for opcion in opciones:
        ratio = fuzz.ratio(nombre, opcion)
        if ratio > max_ratio:
            max_ratio = ratio
            mejor_coincidencia = opcion
    return pd.Series([mejor_coincidencia, max_ratio])

df1[['best_bonarea_name_match', "score_match"]] = df1['norm_product_name'].apply(lambda x: find_best_match(x, df2['bonarea_name']))

df_resultado = pd.merge(df1, df2, left_on='best_bonarea_name_match', right_on='bonarea_name', how='left')

df_resultado = df_resultado[['product_name','norm_product_name', 'best_bonarea_name_match', "score_match",'scrap_price/100g']]


In [232]:
df_resultado = df_resultado.sort_values(by="score_match", ascending=False)
df_resultado

,product_name,norm_product_name,best_bonarea_name_match,score_match,scrap_price/100g
0,Jamón serrano,jamon serrano,jamon serrano,100,1.57
189,JAMON SERRANO,jamon serrano,jamon serrano,100,1.57
141,Jamón serrano,jamon serrano,jamon serrano,100,1.57
142,Jamón serrano gran reserva,jamon serrano gran reserva,jamon serrano gran reserva,100,1.75
1,Jamón serrano familiar,jamon serrano familiar,jamon serrano familiar,100,1.55
...,...,...,...,...,...
263,Jamon cebo iberico,jamon cebo iberico,jamon de cebo iberico 50% raza iberica,64,3.99
66,Jamon iberico cocido,jamon iberico cocido,jamon serrano,55,1.57
155,jamon iberico cocido,jamon iberico cocido,jamon serrano,55,1.57
160,Jamon cocido iberico,jamon cocido iberico,jamon serrano,55,1.57


In [233]:
# keep only first value of duplicated product names
df_resultado = df_resultado.drop_duplicates(subset='product_name', keep='first')

In [234]:
# keep only the ones with a match scoring above 80 %
df_resultado = df_resultado[df_resultado["score_match"] > 80]

In [235]:
# select the product and price columns
df_resultado = df_resultado[["product_name", "scrap_price/100g"]].reset_index(drop=True)

In [236]:
df_resultado

,product_name,scrap_price/100g
0,Jamón serrano,1.57
1,JAMON SERRANO,1.57
2,Jamón serrano gran reserva,1.75
3,Jamón serrano familiar,1.55
4,Finisimo de jamon serrano,1.55
5,Finísimo de jamón serrano,1.55
6,Virutas de jamón serrano,1.42
7,Jamón serrano medias lonchas,1.72
8,jamon serrano,1.57
9,Jamón Serrano Familiar,1.55


In [237]:
# split dataframe by brand bonarea
df_only_bonarea = df_off_spain[df_off_spain["brands"]=="bonarea"].reset_index(drop=True)
df_only_bonarea = df_only_bonarea.drop(columns=["scrap_price/100g"])
df_without_bonarea = df_off_spain[df_off_spain["brands"]!="bonarea"].reset_index(drop=True)

In [240]:
# add the prices to the bonarea dataset
df_only_bonarea = pd.merge(df_only_bonarea, df_resultado, on='product_name', how='left')

,code,countries,product_name,brands,ingredients_text,carbohydrates_100g,energy-kcal_100g,fat_100g,proteins_100g,salt_100g,saturated-fat_100g,sugars_100g,is_na,mono-unsaturated_100g,price_100g,url_producto,url_imagen,norm_product_name,scrap_price/100g
0,00569008500119002642,España,Jamón serrano,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,15.5,30.5,5.1,5.6,0.00,0,9.9,NaN,https://world.openfoodfacts.org/product/005690...,https://images.openfoodfacts.org/images/produc...,jamon serrano,1.57
1,00544714700219006071,España,Jamón serrano familiar,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,15.5,30.5,5.1,5.6,0.00,0,9.9,NaN,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
2,00544717800219006754,España,Jamón serrano familiar,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,16.6,30.5,5.1,5.6,0.00,0,11.0,0.675,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
3,00544719100119006730,España,Jamón serrano familiar,bonarea,,0.5,264.0,15.5,30.5,5.1,5.6,0.00,0,9.9,NaN,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
4,00703619100119003106,en:es,Jamón serrano familiar,bonarea,NaN,0.5,246.0,15.5,30.5,5.1,5.6,0.00,0,9.9,NaN,https://world.openfoodfacts.org/product/007036...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,00569031200122003367,en:es,Jamon serrano,bonarea,NaN,0.5,229.0,11.0,32.0,5.0,3.0,0.00,0,8.0,NaN,https://world.openfoodfacts.org/product/005690...,https://images.openfoodfacts.org/images/produc...,jamon serrano,1.57
246,00597431302422002190,en:es,VIRUTAS DE JAMON SERRANO,bonarea,NaN,0.5,229.0,11.0,32.0,5.0,3.9,0.00,0,7.1,NaN,https://world.openfoodfacts.org/product/005974...,https://images.openfoodfacts.org/images/produc...,virutas de jamon serrano,1.42
247,00569034600122003454,en:es,Jamón serrano / Pernil serrano,bonarea,NaN,0.5,229.0,11.0,32.0,5.0,3.9,0.00,0,7.1,NaN,https://world.openfoodfacts.org/product/005690...,https://images.openfoodfacts.org/images/produc...,jamon serrano / pernil serrano,NaN
248,00428434800122003775,en:es,Jamón Serrano Gran Reserva,bonarea,NaN,0.5,229.0,11.0,32.0,5.0,3.9,0.00,0,7.1,NaN,https://world.openfoodfacts.org/product/004284...,https://images.openfoodfacts.org/images/produc...,jamon serrano gran reserva,1.75


In [241]:
# concat again the splited datasets
result_df = pd.concat([df_only_bonarea, df_without_bonarea], axis=0, ignore_index=True)
result_df

,code,countries,product_name,brands,ingredients_text,carbohydrates_100g,energy-kcal_100g,fat_100g,proteins_100g,salt_100g,saturated-fat_100g,sugars_100g,is_na,mono-unsaturated_100g,price_100g,url_producto,url_imagen,norm_product_name,scrap_price/100g
0,00569008500119002642,España,Jamón serrano,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,15.5,30.5,5.1,5.6,0.0,0,9.9,NaN,https://world.openfoodfacts.org/product/005690...,https://images.openfoodfacts.org/images/produc...,jamon serrano,1.57
1,00544714700219006071,España,Jamón serrano familiar,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,15.5,30.5,5.1,5.6,0.0,0,9.9,NaN,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
2,00544717800219006754,España,Jamón serrano familiar,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,16.6,30.5,5.1,5.6,0.0,0,11.0,0.675,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
3,00544719100119006730,España,Jamón serrano familiar,bonarea,,0.5,264.0,15.5,30.5,5.1,5.6,0.0,0,9.9,NaN,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
4,00703619100119003106,en:es,Jamón serrano familiar,bonarea,NaN,0.5,246.0,15.5,30.5,5.1,5.6,0.0,0,9.9,NaN,https://world.openfoodfacts.org/product/007036...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,8437024100485,en:es,Jamon serrano,NaN,NaN,0.5,238.0,10.0,37.0,4.7,3.6,0.5,0,6.4,NaN,https://world.openfoodfacts.org/product/843702...,https://images.openfoodfacts.org/images/produc...,jamon serrano,NaN
1326,8437013835817,en:es,Jamon de cebo de campo ibérico MONTARAZ,NaN,NaN,1.0,286.0,16.1,34.3,3.9,6.6,0.5,0,9.5,NaN,https://world.openfoodfacts.org/product/843701...,https://images.openfoodfacts.org/images/produc...,jamon de cebo de campo iberico montaraz,NaN
1327,8428204011998,en:es,Jamón serrano reserva,NaN,NaN,0.0,222.0,11.0,30.0,4.6,4.6,0.0,0,6.4,NaN,https://world.openfoodfacts.org/product/842820...,https://images.openfoodfacts.org/images/produc...,jamon serrano reserva,NaN
1328,8431876344141,en:es,Jamón serrano,NaN,NaN,0.2,231.8,10.2,34.7,5.3,3.8,0.2,0,6.4,NaN,https://world.openfoodfacts.org/product/843187...,https://images.openfoodfacts.org/images/produc...,jamon serrano,NaN


In [244]:
# summary
num_total_products = result_df.shape[0]
null_prices_before = result_df["price_100g"].isna().sum()

result_df["price_100g"].fillna(result_df['scrap_price/100g'], inplace=True)
null_prices_after = result_df["price_100g"].isna().sum()
result_df = result_df.drop(columns=["scrap_price/100g"])

print(f"Productos con precio antes de scraping: {num_total_products - null_prices_before}")
print(f"Productos con precio después de scraping: {num_total_products - null_prices_after}")

Productos con precio antes de scraping: 271
Productos con precio después de scraping: 405


C:\Users\oscar\AppData\Local\Temp\ipykernel_10660\789888681.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.57 1.55 1.55 ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result_df["price_100g"].fillna(result_df['scrap_price/100g'], inplace=True)


In [246]:
result_df.to_pickle('df_scraping.pkl')

In [4]:
file_path = 'df_scraping.pkl'
df_off_spain = pd.read_pickle(file_path)

In [5]:
df_off_spain

,code,countries,product_name,brands,ingredients_text,carbohydrates_100g,energy-kcal_100g,fat_100g,proteins_100g,salt_100g,saturated-fat_100g,sugars_100g,is_na,mono-unsaturated_100g,price_100g,url_producto,url_imagen,norm_product_name
0,00569008500119002642,España,Jamón serrano,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,15.5,30.5,5.1,5.6,0.0,0,9.9,1.570,https://world.openfoodfacts.org/product/005690...,https://images.openfoodfacts.org/images/produc...,jamon serrano
1,00544714700219006071,España,Jamón serrano familiar,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,15.5,30.5,5.1,5.6,0.0,0,9.9,1.550,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar
2,00544717800219006754,España,Jamón serrano familiar,bonarea,"Jamón de cerdo, sal, conservantes (nitrito de ...",0.5,264.0,16.6,30.5,5.1,5.6,0.0,0,11.0,0.675,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar
3,00544719100119006730,España,Jamón serrano familiar,bonarea,,0.5,264.0,15.5,30.5,5.1,5.6,0.0,0,9.9,1.550,https://world.openfoodfacts.org/product/005447...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar
4,00703619100119003106,en:es,Jamón serrano familiar,bonarea,NaN,0.5,246.0,15.5,30.5,5.1,5.6,0.0,0,9.9,1.550,https://world.openfoodfacts.org/product/007036...,https://images.openfoodfacts.org/images/produc...,jamon serrano familiar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,8437024100485,en:es,Jamon serrano,NaN,NaN,0.5,238.0,10.0,37.0,4.7,3.6,0.5,0,6.4,NaN,https://world.openfoodfacts.org/product/843702...,https://images.openfoodfacts.org/images/produc...,jamon serrano
1326,8437013835817,en:es,Jamon de cebo de campo ibérico MONTARAZ,NaN,NaN,1.0,286.0,16.1,34.3,3.9,6.6,0.5,0,9.5,NaN,https://world.openfoodfacts.org/product/843701...,https://images.openfoodfacts.org/images/produc...,jamon de cebo de campo iberico montaraz
1327,8428204011998,en:es,Jamón serrano reserva,NaN,NaN,0.0,222.0,11.0,30.0,4.6,4.6,0.0,0,6.4,NaN,https://world.openfoodfacts.org/product/842820...,https://images.openfoodfacts.org/images/produc...,jamon serrano reserva
1328,8431876344141,en:es,Jamón serrano,NaN,NaN,0.2,231.8,10.2,34.7,5.3,3.8,0.2,0,6.4,NaN,https://world.openfoodfacts.org/product/843187...,https://images.openfoodfacts.org/images/produc...,jamon serrano


In [7]:
df_off_spain[df_off_spain["brands"] == "mercadona"]["price_100g"].isna().sum()

50

In [8]:
df_off_spain[df_off_spain["brands"] == "mercadona"][["product_name","price_100g"]]

,product_name,price_100g
309,Jamón de cebo ibérico,NaN
335,Jamón de cebo ibérico,NaN
363,Jamón de bellota ibérico,NaN
366,Jamón serrano,NaN
371,Jamon serrano,3.29
377,Jamón de bellota ibérico,NaN
382,Jamón de bellota ibérico,NaN
387,Jamón de bellota ibérico,NaN
399,Jamón de cebo ibérico,NaN
406,Jamón bellota Ibérico 100% raza,NaN


In [9]:
df_off_spain[df_off_spain["brands"] == "mercadona"]

,code,countries,product_name,brands,ingredients_text,carbohydrates_100g,energy-kcal_100g,fat_100g,proteins_100g,salt_100g,saturated-fat_100g,sugars_100g,is_na,mono-unsaturated_100g,price_100g,url_producto,url_imagen,norm_product_name
309,230582780009406500006115,España,Jamón de cebo ibérico,mercadona,,1.00,397.000000,29.500000,31.910000,3.50,9.70,0.50,0,19.800000,NaN,https://world.openfoodfacts.org/product/230582...,https://images.openfoodfacts.org/images/produc...,jamon de cebo iberico
335,230582780009606500006249,España,Jamón de cebo ibérico,mercadona,,1.00,397.100000,29.500000,31.900000,3.50,9.70,0.50,0,19.800000,NaN,https://world.openfoodfacts.org/product/230582...,https://images.openfoodfacts.org/images/produc...,jamon de cebo iberico
363,230582800010012500012509,Spain,Jamón de bellota ibérico,mercadona,,1.00,393.000000,30.000000,29.500000,3.50,9.00,0.60,0,21.000000,NaN,https://world.openfoodfacts.org/product/230582...,https://images.openfoodfacts.org/images/produc...,jamon de bellota iberico
366,230581470014401680002424,Spain,Jamón serrano,mercadona,,1.00,247.000000,12.000000,33.000000,3.60,5.00,0.50,0,7.000000,NaN,https://world.openfoodfacts.org/product/230581...,https://images.openfoodfacts.org/images/produc...,jamon serrano
371,230582740008603250002809,Spain,Jamon serrano,mercadona,,1.00,299.000000,18.700000,31.800000,3.60,6.80,0.50,0,11.900000,3.29,https://world.openfoodfacts.org/product/230582...,https://images.openfoodfacts.org/images/produc...,jamon serrano
377,230583370008012500010002,Spain,Jamón de bellota ibérico,mercadona,,0.80,350.000000,23.000000,35.000000,4.50,8.90,0.50,0,14.100000,NaN,https://world.openfoodfacts.org/product/230583...,https://images.openfoodfacts.org/images/produc...,jamon de bellota iberico
382,230582800007212500009004,Spain,Jamón de bellota ibérico,mercadona,,1.00,393.000000,30.100000,29.500000,3.50,9.00,0.50,0,21.100000,NaN,https://world.openfoodfacts.org/product/230582...,https://images.openfoodfacts.org/images/produc...,jamon de bellota iberico
387,230583660008512500010634,Spain,Jamón de bellota ibérico,mercadona,,0.10,434.000000,40.800000,17.200000,2.14,8.39,0.10,0,32.410000,NaN,https://world.openfoodfacts.org/product/230583...,https://images.openfoodfacts.org/images/produc...,jamon de bellota iberico
399,230582780010006500006502,España,Jamón de cebo ibérico,mercadona,,1.00,397.000000,29.600000,31.900000,3.50,9.70,0.50,0,19.900000,NaN,https://world.openfoodfacts.org/product/230582...,https://images.openfoodfacts.org/images/produc...,jamon de cebo iberico
406,230584230008415500013022,España,Jamón bellota Ibérico 100% raza,mercadona,,0.50,286.000000,18.000000,33.500000,3.72,6.25,0.50,0,11.750000,NaN,https://world.openfoodfacts.org/product/230584...,https://images.openfoodfacts.org/images/produc...,jamon bellota iberico 100% raza


In [10]:
df_off_spain["brands"].value_counts()

brands
bonarea      250
mercadona     54
              32
carrefour     22
juan luna     17
            ... 
delicato       1
penaranda      1
sodexo         1
frimancha      1
artysan        1
Name: count, Length: 143, dtype: int64